In [ ]:
!pip install finance-datareader
!pip install pycaret

In [40]:
import pandas as pd
import numpy as np
import os
from pycaret.regression import *
import FinanceDataReader as fdr
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from tqdm import tqdm

## Get Stock List

In [43]:

path = ''
list_name = 'stock_list.csv'
sample_name = 'sample_submission.csv'
stock_data = '20210104_20211119_data_interpolate.csv'
stock_test_data = '20210104_1029_pred_1101_05_test.csv'

stock_list = pd.read_csv(os.path.join(path,list_name))
stock_data = pd.read_csv(os.path.join(path,stock_data))
stock_test_data = pd.read_csv(os.path.join(path,stock_test_data))
stock_data['Date'] = pd.to_datetime(stock_data['Date'])
stock_list['종목코드'] = stock_list['종목코드'].apply(lambda x : str(x).zfill(6))


In [121]:
stock_data2 = stock_data[stock_data['Date'] >= start_date]
stock_data2=stock_data2.reset_index()
stock_data2=stock_data2.drop('index',axis=1)
stock_data2['code'] = pd.Series()
stock_data2

,Date,exchange_rate,kospi,kosdaq,kospi100,dow,nasdaq,sp500,미국채10년-2년,BTC,Gold,Oil,US10Y,US2Y,code
0,2021-10-04,1184.12,2990.675,969.285,2941.375,34003.58,14255.5,4300.46,1.2014,60093000.0,33.495,54.445,1.4965,0.27565,NaN
1,2021-10-05,1186.48,2962.170,955.370,2914.050,34315.99,14433.8,4345.72,1.2424,61800000.0,33.500,55.630,1.5280,0.28560,NaN
2,2021-10-06,1190.21,2908.310,922.360,2873.680,34417.98,14501.9,4363.55,1.2305,67121000.0,33.590,54.160,1.5280,0.29750,NaN
3,2021-10-07,1190.88,2959.460,953.430,2920.960,34754.15,14654.0,4399.76,1.2665,65474000.0,33.400,55.360,1.5760,0.30950,NaN
4,2021-10-08,1196.79,2956.300,953.110,2914.790,34746.71,14579.5,4391.36,1.2922,65982000.0,33.420,55.470,1.6120,0.31980,NaN
5,2021-10-11,1195.30,2936.340,946.630,2889.125,34496.85,14486.2,4361.19,1.2743,67596500.0,33.470,55.790,1.5920,0.33085,NaN
6,2021-10-12,1197.29,2916.380,940.150,2863.460,34378.08,14465.9,4350.64,1.2301,69211000.0,33.520,56.110,1.5720,0.34190,NaN
7,2021-10-13,1186.18,2944.410,953.470,2888.460,34378.60,14571.6,4363.80,1.1820,69925000.0,34.110,56.160,1.5420,0.36000,NaN
8,2021-10-14,1181.69,2988.640,983.430,2930.290,34911.70,14823.4,4438.23,1.1518,69753000.0,34.200,56.810,1.5160,0.36420,NaN
9,2021-10-15,1182.07,3015.060,990.540,2957.370,35295.48,14897.3,4471.37,1.1791,75707000.0,33.660,57.150,1.5740,0.39490,NaN


In [122]:
start_date = '20211004'
end_date = '20211105'

In [123]:

stock_data3 = pd.DataFrame()

Business_days = pd.DataFrame(pd.date_range(start_date,end_date,freq='B'), columns = ['Date'])
i=0
for code in tqdm(stock_list['종목코드'].values):
  i+=1
  if i ==5:
    break
  
  stock_data2['code']=stock_data2['code'].fillna(code)
  data = fdr.DataReader(code, start = start_date, end = end_date)[['Close']].reset_index()
  stock_data2 = pd.merge(stock_data2, data)
  stock_data3 = pd.concat([stock_data3, stock_data2])
  stock_data2['code']=stock_data2.drop('code',axis=1)
  stock_data2['code'] = pd.Series()

  1%|          | 4/370 [00:06<10:33,  1.73s/it]


In [124]:
stock_data3

,Date,exchange_rate,kospi,kosdaq,kospi100,dow,nasdaq,sp500,미국채10년-2년,BTC,Gold,Oil,US10Y,US2Y,code,Close
0,2021-10-05,1186.48,2962.17,955.37,2914.05,34315.99,14433.8,4345.72,1.2424,61800000.0,33.50,55.63,1.528,0.2856,005930,72200
1,2021-10-06,1190.21,2908.31,922.36,2873.68,34417.98,14501.9,4363.55,1.2305,67121000.0,33.59,54.16,1.528,0.2975,005930,71300
2,2021-10-07,1190.88,2959.46,953.43,2920.96,34754.15,14654.0,4399.76,1.2665,65474000.0,33.40,55.36,1.576,0.3095,005930,71600
3,2021-10-08,1196.79,2956.30,953.11,2914.79,34746.71,14579.5,4391.36,1.2922,65982000.0,33.42,55.47,1.612,0.3198,005930,71500
4,2021-10-12,1197.29,2916.38,940.15,2863.46,34378.08,14465.9,4350.64,1.2301,69211000.0,33.52,56.11,1.572,0.3419,005930,69000
5,2021-10-13,1186.18,2944.41,953.47,2888.46,34378.60,14571.6,4363.80,1.1820,69925000.0,34.11,56.16,1.542,0.3600,005930,68800
6,2021-10-14,1181.69,2988.64,983.43,2930.29,34911.70,14823.4,4438.23,1.1518,69753000.0,34.20,56.81,1.516,0.3642,005930,69400
7,2021-10-15,1182.07,3015.06,990.54,2957.37,35295.48,14897.3,4471.37,1.1791,75707000.0,33.66,57.15,1.574,0.3949,005930,70100
8,2021-10-18,1185.47,3006.68,993.86,2947.54,35259.01,15021.8,4486.48,1.1697,76494000.0,33.57,56.96,1.591,0.4213,005930,70200
9,2021-10-19,1175.13,3029.04,1005.35,2975.17,35458.37,15129.1,4519.63,1.2416,78715000.0,33.68,57.42,1.641,0.3994,005930,70600


In [73]:
train = stock_data3[stock_data3['Date'] < '2021-11-01']
test =  stock_test_data

In [74]:
train.head()

,Date,exchange_rate,kospi,kosdaq,kospi100,dow,nasdaq,sp500,미국채10년-2년,BTC,Gold,Oil,US10Y,US2Y,code,Close
0,2021-06-01,1108.38,3221.87,984.59,3253.69,34575.3,13736.5,4202.04,1.4574,4.3208e+07,36.2,46.33,1.606,0.1486,005930,80600
1,2021-06-02,1110.92,3224.23,981.1,3249.1,34600.4,13756.3,4208.12,1.4424,4.372e+07,36.37,46.9,1.589,0.1466,005930,80800
2,2021-06-03,1116.46,3247.43,990.19,3278.28,34577,13614.5,4192.85,1.4666,4.544e+07,35.64,46.97,1.625,0.1584,005930,82800
3,2021-06-04,1110.52,3240.08,987.58,3264.07,34756.4,13814.5,4229.89,1.4083,4.2973e+07,36.04,47.26,1.557,0.1487,005930,82200
4,2021-06-07,1110.62,3252.12,985.86,3268.83,34630.2,13881.7,4226.52,1.4123,3.8953e+07,36.19,47.27,1.569,0.1567,005930,81900


In [75]:
test.head()

,Date,weekday,weeknum,kosdaq,nasdaq,sp500,dow,kospi
0,2021-11-01,0,44,1006.418941,14737.349456,4407.581144,33977.188,3062.021995
1,2021-11-02,1,44,1007.549998,14770.291740,4428.330825,34165.137,3064.841639
2,2021-11-03,2,44,1006.637302,14768.971554,4428.732729,34169.477,3062.021995
3,2021-11-04,3,44,1007.108948,14793.617235,4452.543789,34432.043,3059.876766
4,2021-11-05,4,44,1007.500128,14834.681366,4457.262093,34424.100,3058.086760


In [ ]:
  model=setup(
        data=train,
        target = 'Close',
        log_experiment = True,
        normalize = True,

        silent=True
      )
  tuned_best3models = tune_model(
      compare_models(sort='MAPE', n_select=3)
      )
  tuned_model_ = finalize_model(tuned_model)
  
  pred = predict_model(tuned_model_, test)

In [ ]:

# 
# code = '005930'
# data = fdr.DataReader(code, start = start_date, end = end_date)[['Close']].reset_index()
# 
# data['weekday'] = data.Date.apply(lambda x : x.weekday())
# data['weeknum'] = data.Date.apply(lambda x : x.strftime('%V'))
# data.Close = data.Close.interpolate()


In [ ]:
# merge_stock_data.to_datetime()
# data = pd.merge(data, merge_stock_data, on='Date')

In [ ]:
# train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 215 entries, 0 to 214
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Date           215 non-null    datetime64[ns]
 1   exchange_rate  215 non-null    float64       
 2   kospi          215 non-null    float64       
 3   kosdaq         215 non-null    float64       
 4   kospi100       215 non-null    float64       
 5   dow            215 non-null    float64       
 6   nasdaq         215 non-null    float64       
 7   sp500          215 non-null    float64       
 8   미국채10년-2년      215 non-null    float64       
 9   BTC            215 non-null    float64       
 10  Gold           215 non-null    float64       
 11  Oil            215 non-null    float64       
 12  US10Y          215 non-null    float64       
 13  US2Y           215 non-null    float64       
 14  Close          204 non-null    float64       
dtypes: datetime64[ns](1), f

In [ ]:

# features = ['Date','kospi','kosdaq','kospi100','dow','nasdaq','sp500','exchange_rate','미국채10년-2년','BTC', 'US2Y','US10Y','Gold','Oil']
# train = train[features]

# stock_pred_dict = {} 
# for i in range(len(features)-1):
#   model=setup(
#       data=train,
#       target = features[i+1],
#       ignore_features = features[i+2:],
#       log_experiment = True,
#       normalize = True,
#       silent=True
#     )
#   tuned_model = tune_model(compare_models(sort ='MAPE', n_select=1))
#   tuned_model_ = finalize_model(tuned_model)
#   pred = predict_model(tuned_model_, test)
#   test[features[i+1]] = pred['Label']


,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,0.9570,1.3614,1.1668,0.9180,0.0258,0.0214
1,0.8931,1.7718,1.3311,0.9504,0.0311,0.0211
2,1.0833,2.5151,1.5859,0.9338,0.0354,0.0252
3,0.7208,0.9219,0.9602,0.9692,0.0207,0.0160
4,0.5455,0.6187,0.7866,0.9801,0.0179,0.0127
5,1.0614,1.7742,1.3320,0.9334,0.0285,0.0234
6,0.9910,1.3095,1.1443,0.9414,0.0245,0.0219
7,0.9363,1.4049,1.1853,0.9520,0.0236,0.0189
8,1.1191,1.5377,1.2400,0.9283,0.0274,0.0253
9,0.5719,0.4818,0.6941,0.9834,0.0148,0.0125


In [ ]:

for code in tqdm(stock_list['종목코드'].values):
  code_data = fdr.DataReader(code, start = start_date, end = end_date)[['Close']].reset_index()
  train['Close'] = code_data['Close']

  stock_pred_dict[code] = pred['Label']


,Description,Value
0,session_id,3801
1,Target,Close
2,Original Data,"(215, 15)"
3,Missing Values,False
4,Numeric Features,13
5,Categorical Features,0
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(150, 30)"


IntProgress(value=0, description='Processing: ', max=94)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,12:50:58
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Ridge Regression


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lasso,Lasso Regression,524.0168,452137.1601,656.7441,0.8593,0.0546,0.0434,0.018
lr,Linear Regression,527.0477,463601.4452,666.4990,0.8564,0.0556,0.0438,0.014


In [ ]:
test

In [ ]:
train

- 예측

- 실제 Public 값